## Embed validation and test sets of individually annotated Ovenbird songs with BirdNET

We include an Anaconda environment file `bmz_birdnet_env.yml` since BirdNET may require different package dependency configurations than other models. 

In [ ]:
import bioacoustics_model_zoo as bmz
import pandas as pd
import numpy as np
from pathlib import Path

load set of clips to predict on for validation and test sets

In [ ]:
annotated_dataset_path = "../../../localization_dataset_labeled_clips"
labels = pd.read_csv(f"{annotated_dataset_path}/labeled_clips.csv")
val = labels[labels["data_split"] == "val"]
test = labels[labels["data_split"] == "test"]

# only predict on the 4-7 s part of the 10s clip
val = pd.DataFrame({"file": val["rel_path"], "start_time": 4, "end_time": 7}).set_index(
    ["file", "start_time", "end_time"]
)
test = pd.DataFrame(
    {"file": test["rel_path"], "start_time": 4, "end_time": 7}
).set_index(["file", "start_time", "end_time"])
len(val), len(test)

(2739, 1224)

load BirdNET from the bioacoustics model zoo

In [ ]:
birdnet = bmz.BirdNET()

File BirdNET_GLOBAL_6K_V2.4_Labels_af.txt already exists; skipping download.
downloading model from URL...
File BirdNET_GLOBAL_6K_V2.4_Model_FP16.tflite already exists; skipping download.


/Users/SML161/miniconda3/envs/opso0110/lib/python3.11/site-packages/opensoundscape/ml/cnn.py:599: UserWarning: 
                    This architecture is not listed in opensoundscape.ml.cnn_architectures.ARCH_DICT.
                    It will not be available for loading after saving the model with .save() (unless using pickle=True). 
                    To make it re-loadable, define a function that generates the architecture from arguments: (n_classes, n_channels) 
                    then use opensoundscape.ml.cnn_architectures.register_architecture() to register the generating function.

                    The function can also set the returned object's .constructor_name to the registered string key in ARCH_DICT
                    to avoid this warning and ensure it is reloaded correctly by opensoundscape.ml.load_model().

                    See opensoundscape.ml.cnn_architectures module for examples of constructor functions
                    
  warnings.warn(
/Users/SML161/min

generate feature vector embeddings

In [ ]:
test_embeddings = birdnet.embed(
    test, batch_size=128, num_workers=0, audio_root=annotated_dataset_path
)
val_embeddings = birdnet.embed(
    val, batch_size=128, num_workers=0, audio_root=annotated_dataset_path
)
# test_embeddings.to_csv(f"../resources/birdnet_test_embeddings.csv")
# val_embeddings.to_csv(f"../resources/birdnet_val_embeddings.csv")

Path("../resources/precomputed_embeddings").mkdir(exist_ok=True)
np.save(
    f"../resources/precomputed_embeddings/birdnet_test_embeddings.npy",
    test_embeddings.values,
)
np.save(
    f"../resources/precomputed_embeddings/birdnet_val_embeddings.npy",
    val_embeddings.values,
)

  0%|          | 0/10 [00:00<?, ?it/s]

/Users/SML161/miniconda3/envs/opso0110/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/22 [00:00<?, ?it/s]

/Users/SML161/miniconda3/envs/opso0110/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
